In [52]:
# https://towardsdatascience.com/simple-transformers-named-entity-recognition-with-transformer-models-c04b9242a2a0

In [53]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [54]:
import pandas as pd
data = pd.read_csv("address_training_bert_1.csv",encoding="latin1" )

In [55]:
data.head(30)

,Sentence#,Word,Tag
0,0,3944,HOUSE_NO
1,0,Ranch,STREET_NAME
2,0,Road,STREET_NAME
3,0,620,STREET_NAME
4,0,S,STREET_NAME
5,0,Bldg,SUITE_NO
6,0,G2,SUITE_NO
7,0,Ste,SUITE_NO
8,0,120,SUITE_NO
9,1,1650,HOUSE_NO


In [56]:
data.rename(columns={"Sentence#":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [57]:
data["labels"] = data["labels"].str.upper()

In [58]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [59]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [60]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [61]:
train_data

,sentence_id,words,labels
638,159,Zetland,STREET_NAME
403,101,Kolejowa,STREET_NAME
166,32,1750,HOUSE_NO
80,13,105,SUITE_NO
7,0,Ste,SUITE_NO
...,...,...,...
547,138,SE,STREET_NAME
375,91,Batai,STREET_NAME
101,17,NE,STREET_NAME
348,83,9,HOUSE_NO


In [62]:
from simpletransformers.ner import NERModel,NERArgs
# https://simpletransformers.ai/docs/ner-model/

In [63]:
label = data["labels"].unique().tolist()
label

['HOUSE_NO', 'STREET_NAME', 'SUITE_NO']

In [64]:
args = NERArgs()
args.num_train_epochs = 3
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 16
args.eval_batch_size = 16

In [65]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args,  use_cuda=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
model.train_model(
    train_data,
    eval_data = test_data,
    acc=accuracy_score,
    # output_dir='grocery_model'
)

Epoch 3 of 3: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:27<00:00, 49.33s/it]


(36, 0.4844694265888797)

In [67]:
result, model_outputs, preds_list = model.eval_model(test_data)

Running Evaluation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:09<00:00,  1.17s/it]
/home/rajaraman/miniconda3/envs/dl39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: STREET_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/rajaraman/miniconda3/envs/dl39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HOUSE_NO seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/rajaraman/miniconda3/envs/dl39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SUITE_NO seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [68]:
result

{'eval_loss': 0.3541808146983385,
 'precision': 0.8343949044585988,
 'recall': 0.8291139240506329,
 'f1_score': 0.8317460317460317}

In [50]:
prediction, model_output = model.predict(["254 Spadina Road"])

Running Prediction: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.50it/s]


In [51]:
prediction

[[{'254': 'STREET_NAME'}, {'Spadina': 'STREET_NAME'}, {'Road': 'STREET_NAME'}]]